In [97]:
import pandas as pd

In [98]:
df=pd.read_excel('IA3-2.xlsx')

In [99]:
df.head()

,id,review,label
0,1,About the shop: There is a restaurant in Soi L...,restaurant
1,2,About the shop: Through this store for about t...,restaurant
2,3,Roast Coffee &amp; Eatery is a restaurant loca...,restaurant
3,4,Eat from the children. The shop is opposite. P...,restaurant
4,5,The Ak 1 shop at another branch tastes the sam...,restaurant


In [100]:
df_review=df['review']

1) Transform  those  reviews  into  a  term-document  matrix,  lemmatize  all  the  words,  remove  the  
stop-words  and  punctuations,  set  the  minimal  document  frequency  for  each  term  to  be  5  and  
include 2-gram. 

In [273]:
#lemmatizing the words, removing stop-words and punctuations

import nltk
from nltk.corpus import stopwords

lemmatizer = nltk.stem.WordNetLemmatizer()

processed_df = []
for doc in df_review:
    tokens = nltk.word_tokenize(doc.lower())
    tokens = [lemmatizer.lemmatize(token) for token in tokens if token.isalpha()]
    tokens = [token for token in tokens if not token in stopwords.words('english')]
    processed_df.append(" ".join(tokens))
print(processed_df)

['shop restaurant soi langsuan road inside luxx hotel restaurant decorated rustic style white wall glass table red chair parquet floor dim light open night romantic atmosphere duck l orange pork wellington french onion soup average price baht food review duck l orange duck breast sliced pork wellington piece tender pork stuffed stuffing wrapped thin pastry baked serve sauce duck roll duck wrapped vegetable dough wrap much delicious french onion soup sweet taste garnish cheese bake scallop scallop thick cream cheese serve thin toast eat together score topic lt atmosphere gt atmosphere nice romantic lt food taste gt lt service gt good service good food recommendation lt value gt price quite expensive baht ordered original special conclusion atmosphere good delicious food three half star delicious food note review personal use like please forgive think one comment win competition good restaurant thank', 'shop store three year first time try restaurant located soi sansabai sukhumvit middle

In [331]:
# The result from previous step is vectorized and the term document matrix is obtained

from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range = (1, 2), min_df = 5)
X = vectorizer.fit_transform(processed_df)
print(X.toarray())


[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.13699221 ... 0.         0.         0.        ]]


2) Use the LDA model to extract the topics of each document. In particular, we assume there are 6 
topics.  

In [400]:
# Creating a vector using Countvectorizer to create LDA models
vectorizer1 = CountVectorizer()
X2 = vectorizer1.fit_transform(processed_df)
terms = vectorizer1.get_feature_names()

In [401]:
from sklearn.decomposition import LatentDirichletAllocation
lda = LatentDirichletAllocation(n_components = 6).fit(X2) # learn two probilities
#terms = vectorizer.get_feature_names()

In [402]:
for topic_idx, topic in enumerate(lda.components_):
    print("Topic %d:" % (topic_idx))
    print(" ".join([terms[i] for i in topic.argsort()[:-4-1:-1]]))

Topic 0:
quot film ha also
Topic 1:
gt good price lt
Topic 2:
quot wa love people
Topic 3:
good eat delicious like
Topic 4:
quot film wa ha
Topic 5:
water food zhang oil


3)  Report the topic distribution and the top-2 topics of the first 10 restaurant reviews (id = [1:10]) 
and the first 10 movie reviews (id = [501:510]).

In [435]:
#Extracting the first 10 restaurant reviews (id = [1:10]) and converting to array 

res_subset = (X2.toarray()[0:10])
print(res_subset)
#res_subset.shape

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [404]:
##Find the probability of a document containing a topic (topic distribution)

res_subset_trans=lda.transform(X2[0])
res_subset_trans

array([[0.0012012 , 0.3001265 , 0.00120264, 0.69506343, 0.00120246,
        0.00120377]])

In [422]:
##identifying top-2 topics of the first 10 restaurant reviews 

lda_rest = LatentDirichletAllocation(n_components=6).fit(res_subset)

top_topic_indices= lda_rest.transform(res_subset).argsort()[:,:-1-1:-1]
# Get the top two topic names for each document
top_topic_names = [[f"Topic {idx}" for idx in doc_topics] for doc_topics in top_topic_indices]

# Print the top two topic names for each document
for i, doc_topics in enumerate(top_topic_names):
    print(f"Document {i+1}: {doc_topics}")

Document 1: ['Topic 5']
Document 2: ['Topic 5']
Document 3: ['Topic 5']
Document 4: ['Topic 2']
Document 5: ['Topic 4']
Document 6: ['Topic 5']
Document 7: ['Topic 2']
Document 8: ['Topic 5']
Document 9: ['Topic 5']
Document 10: ['Topic 2']


From the above output we can see that for the first ten restaurant reviews in the range [1:10], 
topics 5 and 2 are the mostly applciable to these documents

In [436]:
#Extracting the first 10 movie reviews (id = [501:511]) and converting to array 

movie_subset = (X2.toarray()[500:510])
print(movie_subset)
#movie_subset.shape

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [424]:
##Find the probability of a document containing a topic (topic distribution)


movie_subset_trans=lda.transform(X2[500])
movie_subset_trans

array([[0.00129896, 0.07259452, 0.92220253, 0.00130073, 0.00130414,
        0.00129912]])

In [433]:
##identifying top-2 topics of the first 10 movie reviews 

lda_rest = LatentDirichletAllocation(n_components=6).fit(movie_subset)

top_movie_indices= lda_rest.transform(movie_subset).argsort()[:,:-1-1:-1]
# Get the top two topic names for each document
top_topic_names = [[f"Topic {idx}" for idx in doc_topics] for doc_topics in top_movie_indices]

# Print the top two topic names for each document
for i, doc_topics in enumerate(top_topic_names):
    print(f"Document {i+1}: {doc_topics}")

Document 1: ['Topic 4']
Document 2: ['Topic 4']
Document 3: ['Topic 0']
Document 4: ['Topic 0']
Document 5: ['Topic 0']
Document 6: ['Topic 0']
Document 7: ['Topic 0']
Document 8: ['Topic 0']
Document 9: ['Topic 0']
Document 10: ['Topic 0']


From the above output we can see that for the first ten movie reviews in the range [500:510], topics 4 and 0 are the mostly applciable to these documents

4)  Find  the  top-5  terms  (terms  with  the  top-5  highest  weights)  for  each  of  the 6  topics.  Based  on  
those terms, describe what those topics are about.

In [434]:
for i, topic_weights in enumerate(lda.components_):
    top_terms_indices = topic_weights.argsort()[::-1][:5] # Get the indices of the top 5 terms
    top_terms = [terms[index] for index in top_terms_indices] # Get the top 5 terms
    print(f"Topic {i}: {', '.join(top_terms)}")

Topic 0: quot, film, ha, also, drama
Topic 1: gt, good, price, lt, sushi
Topic 2: quot, wa, love, people, like
Topic 3: good, eat, delicious, like, taste
Topic 4: quot, film, wa, ha, people
Topic 5: water, food, zhang, oil, eat


 From the above result we can see that: 
topic 0 - belongs to films/movie review category 
topic 1 - belongs to restaurant review category
topic 2 - belongs to films/movie review category 
topic 3 - belongs to restaurant review category
topic 4 - belongs to films/movie review category
topic 5 - belongs to restaurant review category


5) Based on finding in 3 and 4, describe what review 1 [ID=1] and review 501 [ID=501] are about? 

From the results of #3 and #4 we can infer that

the review no [1] most likely belongs to "Restaurant" category since the topic is related to eat and delicious
the review no [501] most likely belongs to "Films" category since the topic is related to film and people
